In [276]:
import pandas as pd
import requests as req
from json import loads, dumps
import time 
import uuid

# turn off column width
#pd.set_option('display.max_colwidth', -1)

In [137]:
cols = ["country", "state", "city", "fbURL", "gmapsURL", "gmapsJSON"]
campsites = pd.read_csv('campsitesLocationInfo2.csv', sep="|", header=None)
campsites.columns = cols

In [138]:
#no null values
campsites.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109 entries, 0 to 1108
Data columns (total 6 columns):
country      1109 non-null object
state        1109 non-null object
city         1109 non-null object
fbURL        1109 non-null object
gmapsURL     1109 non-null object
gmapsJSON    1109 non-null object
dtypes: object(6)
memory usage: 60.6+ KB


In [139]:
# some duplcaite locations (gmapsJSON) and fb URLS
# NOTE: duplicate city names are okay and expected
campsites.describe()

,country,state,city,fbURL,gmapsURL,gmapsJSON
count,1109,1109,1109,1109,1109,1109
unique,142,86,1071,1099,1109,1107
top,United States,none,Columbia,https://www.facebook.com/groups/free.code.camp.northern.arizona/,"https://maps.googleapis.com/maps/api/geocode/json?address=Bristol,England","{ ""results"" : [ { ""address_components"" : [ { ""long_name"" : ""Hull"", ""short_name"" : ""Hull"", ""types"" : [ ""locality"", ""political"" ] }, { ""long_name"" : ""England"", ""short_name"" : ""England"", ""types"" : [ ""administrative_area_level_1"", ""political"" ] }, { ""long_name"" : ""United Kingdom"", ""short_name"" : ""GB"", ""types"" : [ ""country"", ""political"" ] } ], ""formatted_address"" : ""Hull, UK"", ""geometry"" : { ""bounds"" : { ""northeast"" : { ""lat"" : 53.8132502, ""lng"" : -0.2413964 }, ""southwest"" : { ""lat"" : 53.71951499999999, ""lng"" : -0.4225751 } }, ""location"" : { ""lat"" : 53.7456709, ""lng"" : -0.3367413 }, ""location_type"" : ""APPROXIMATE"", ""viewport"" : { ""northeast"" : { ""lat"" : 53.8132502, ""lng"" : -0.2413964 }, ""southwest"" : { ""lat"" : 53.71951499999999, ""lng"" : -0.4225751 } } }, ""place_id"" : ""ChIJrWABfI8g1kcR_zDxgp1-RG0"", ""types"" : [ ""locality"", ""political"" ] } ], ""status"" : ""OK""}"
freq,359,651,3,4,1,2


In [140]:
print 'here are some duplicates facebook groups:'
dupFB = campsites[campsites['fbURL'].duplicated(False)]
print dupFB[['city', 'fbURL']]
print "these looks intentional"

here are some duplicates facebook groups:
              city  \
560   Gdańsk         
568   Sopot          
739   Flagstaff      
741   Phoenix        
742   Prescott       
743   Scottsdale     
744   Sedona         
746   Tempe          
748   Verde Valley   
752   Alameda        
756   Berkeley       
782   Oakland        
854   Centralia      
859   Peoria         
862   Washington     
1067  Centralia      

                                                                         fbURL  
560   https://www.facebook.com/groups/free.code.camp.trojmiasto/                
568   https://www.facebook.com/groups/free.code.camp.trojmiasto/                
739   https://www.facebook.com/groups/free.code.camp.northern.arizona/          
741   https://www.facebook.com/groups/free.code.camp.tempe.phoenix.scottsdale/  
742   https://www.facebook.com/groups/free.code.camp.northern.arizona/          
743   https://www.facebook.com/groups/free.code.camp.tempe.phoenix.scottsdale/  
744   https://ww

In [141]:
print 'here are some duplicates google map responses:'
dupLocations = campsites['gmapsJSON'].duplicated(False)
print campsites[dupLocations]
print "It looks like:"
print "-Kingston upon Hull and Hull are the same place according to google maps"
print "-Women only group didn't have a city"
print "-Hampton Roads is in Virginia"
print "NOTE: Dropping these for now, but could be handled better"
campsites = campsites[~dupLocations]

here are some duplicates google map responses:
           country      state                city  \
221  England        none       Hull                 
223  England        none       Kingston upon Hull   
628  Saudi Arabia   none       Women only group     
730  United States  Ambiguous  Hampton Roads        

                                                                  fbURL  \
221  https://www.facebook.com/groups/free.code.camp.hull/                 
223  https://www.facebook.com/groups/free.code.camp.kingston.upon.hull/   
628  https://www.facebook.com/groups/girlscodeksa/                        
730  https://www.facebook.com/groups/free.code.camp.hamptonroads/         

                                                                                            gmapsURL  \
221  https://maps.googleapis.com/maps/api/geocode/json?address=Hull,England                            
223  https://maps.googleapis.com/maps/api/geocode/json?address=Kingston+upon+Hull,England              

In [84]:
# MODEL FOR POSTING 
## {
##   "id": "string",
##   "url": "string",
##   "createdByUsername": "string",
##   "createdAt": "2016-03-12",
##   "lastUpdatedAt": "2016-03-12",
##   "isApproved": false,
##   "isDeleted": false,
##   "city": "string",
##   "subdivision": "string",
##   "country": "string",
##   "location": "string",
##   "mapURL": "string",
##   "googleId": "string"
## }

In [288]:
parsedLocations = campsites
# build list of dicts to create
toCreate = []
for index, row in parsedLocations.iterrows():
    createDict = {}
    # parse info out of dataframe
    createDict['city'] = row['city']
    createDict['subdivision'] = row['state']
    createDict['country'] = row['country']
    createDict['url'] = row['fbURL']
    # load gmaps json
    gmapsInfo = loads(row['gmapsJSON'])['results'][0]
    loc = gmapsInfo['geometry']['location']
    createDict['location'] = str(loc['lat']) + ',' + str(loc['lng'])
    createDict['googleId'] = str(gmapsInfo['place_id'])
    # NOTE: mapURL is missing and requires another lookup :/
    
    # add other info
    now = time.strftime('%c')
    createDict['id'] = uuid.uuid4().hex
    createDict['createdByUsername'] = 'BULKCAMPSITEUPLOAD'
    createDict['createdAt'] = now
    createDict['lastUpdatedAt'] = now
    createDict['isApproved'] = False # requires approval
    createDict['isDeleted'] = False 
    # append json
    toCreate.append(createDict)
    

In [273]:
# write out json to file 
with open('toWrite.json', 'w') as fp:
    for campsite in toCreate:
        data = dumps(campsite)
        fp.write(data + "\n")

'{"city": "Tirana", "subdivision": "none", "isApproved": false, "url": "https://www.facebook.com/groups/free.code.camp.tirana/", "country": "Albania", "isDeleted": false, "createdAt": "Sat Mar 12 18:13:58 2016", "lastUpdatedAt": "Sat Mar 12 18:13:58 2016", "location": "41.3275459,19.8186982", "createdByUsername": "BULKCAMPSITEUPLOAD", "id": "4ee90a2b19ef439089359d210b8bb467", "googleId": "ChIJ28X6cAQxUBMRIDdlEK-SAAQ"}'